<a href="https://colab.research.google.com/github/pjot726/PyTorchTutorials/blob/main/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Here, we import needed packages from PyTorch. torchvision.datasets contains the datasets of interest, and DataLoader wraps an iterable around the dataset.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

We now download the training and test data from the open datasets.

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We now pass this dataset to the DataLoader object, which wraps an iterable over the dataset. By specifying a batch size of 64, the element of the iterable will all be 64 features and labels.

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Now, we create our neural network.

In [5]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


We now begin training the model, and to do so we define a loss function and an optimizer.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

We now define the training function, where we make predictions on the training set and backpropagates the prediction error to adjust model parameters.

In [7]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We can also write a function to test the model's performance against the test set.

In [8]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We now train the model.

In [9]:
epochs = 5
for t in range(epochs):
  print(f"Epoch: {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch: 1
-------------------------------
loss: 2.303605 [   64/60000]
loss: 2.291878 [ 6464/60000]
loss: 2.273589 [12864/60000]
loss: 2.271310 [19264/60000]
loss: 2.252773 [25664/60000]
loss: 2.223415 [32064/60000]
loss: 2.233103 [38464/60000]
loss: 2.200270 [44864/60000]
loss: 2.198922 [51264/60000]
loss: 2.174995 [57664/60000]
Test Error: 
 Accuracy: 47.3%, Avg loss: 2.171097 

Epoch: 2
-------------------------------
loss: 2.179066 [   64/60000]
loss: 2.173586 [ 6464/60000]
loss: 2.120617 [12864/60000]
loss: 2.140170 [19264/60000]
loss: 2.100584 [25664/60000]
loss: 2.033632 [32064/60000]
loss: 2.061440 [38464/60000]
loss: 1.985260 [44864/60000]
loss: 1.983873 [51264/60000]
loss: 1.934128 [57664/60000]
Test Error: 
 Accuracy: 58.4%, Avg loss: 1.931042 

Epoch: 3
-------------------------------
loss: 1.947848 [   64/60000]
loss: 1.930828 [ 6464/60000]
loss: 1.816123 [12864/60000]
loss: 1.863884 [19264/60000]
loss: 1.778480 [25664/60000]
loss: 1.705666 [32064/60000]
loss: 1.732739 [384